In [1]:
from gevent import monkey
monkey.patch_all()
from gevent.queue import Queue
import gevent

import sys
sys.path.append('/Users/hzm/Desktop/YangTeng/code/venv/code/00.module')
import Proxy
import UA
import time

# = = = = = = = = = = = = = = = = = =

import pandas as pd
from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=' + Proxy.get_Proxy_selenium())
option.add_argument('user-agent=' + UA.get_UA())
option.page_load_strategy = 'eager'

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./detail.xlsx', header=0).fillna('')
list_vehicle_url = df_menu['vehicle_url'].to_list()

print('总数量：' + str(len(list_vehicle_url)))
print()

df = pd.DataFrame()

# = = = = = = = = = = = = = = = = = =

work = Queue()
for url in list_vehicle_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(61):
            if url == None or url == '':
                break
            
            try:
                browser = Chrome(service=Service('../../../chromedriver'), options=option)
                browser.maximize_window()
                
                # = = = = = = = = = = = = = = = = = =

                browser.get(url)
                
                time.sleep(3)
                
                # = = = = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                while True:
                    wait = WebDriverWait(browser, 60)
                    wait.until(EC.presence_of_element_located((By.XPATH, '//tr[@class="detail-app-row"]')))
                    list_row = browser.find_elements(by=By.XPATH, value='//tr[@class="detail-app-row"]')

                    for row in list_row:
                        make = row.find_element(by=By.XPATH, value='./td[2]').text.strip()
                        model = row.find_element(by=By.XPATH, value='./td[3]').text.strip()
                        year = row.find_element(by=By.XPATH, value='./td[1]').text.strip()

                        if make not in dict_vehicle:
                            dict_vehicle[make] = {model: [year]}
                        else:
                            if model not in dict_vehicle[make]:
                                dict_vehicle[make][model] = [year]
                            else:
                                dict_vehicle[make][model].append(year)

                    list_next = browser.find_elements(by=By.XPATH, value='//a[@id="pagingBottom_nextButton"]')
                    if len(list_next) == 0:
                        break
                    else:
                        list_next[0].click()

                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        list_year = list(set(dict_vehicle[make][model]))
                        if len(list_year) == 1:
                            list_vehicle.append(make + ' ' + model + ' ' + list_year[0])
                        else:
                            list_vehicle.append(make + ' ' + model + ' ' + min(list_year) + '-' + max(list_year))

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = = = = =
                
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'vehicle_url': url,
                                         'Vehicle': vehicle}])
                        
                browser.quit()
                
                status = 'ok'

                break

            except:
                time.sleep(3)
                continue

        # = = = = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'vehicle_url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')
        
        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('输出中...')
df.to_excel('./vehicle.xlsx', index=False)
print()
print('搞定！')

总数量：25

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=76754&PartType=HVAC%20Control%20Module  <->  [ok] - 剩余数量：5
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=50180&PartType=HVAC%20Control%20Module  <->  [ok] - 剩余数量：4
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=74181&PartType=HVAC%20Control%20Module  <->  [ok] - 剩余数量：3
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=74179&PartType=HVAC%20Control%20Module  <->  [ok] - 剩余数量：2
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=49317&PartType=HVAC%20Control%20Module  <->  [ok] - 剩余数量：1
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=82894&PartType=HVAC%20Control%20Module  <->  [ok] - 剩余数量：0
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=65302&PartType=HVAC%20Control%20Module  <->  [ok] - 剩余数量：0
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=76741&PartType=HVAC%20Control%20Module  <->  [ok] - 剩余数量：0
https://www.dormanproducts.com/itemdetai

KeyboardInterrupt
2023-08-23T01:32:02Z


KeyboardInterrupt: 